# Train model with different tools

## load data

In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hd_ttH=pandas.read_hdf('../../Files/skimmed/ttH.h5', 'df')
hd_ttW=pandas.read_hdf('../../Files/skimmed/ttW.h5', 'df')

In [3]:
hd_ttH.head()
#hd_ttH['DRll01']

,DEtall01,lep_flavour,max_eta,minDeltaR_LJ_0,minDeltaR_LJ_1,Meff,DRlj00,min_DRl0b,min_DRlj_new,DPhij0MET,...,DRll01,nJets_OR_T,nJets_OR_T_MV2c10_70,HT_lep,HT_jets,lead_jetPt,sublead_jetPt,lepSFTrigTight,lepSFObjTight,weightS
entry,,,,,,,,,,,,,,,,,,,,,
0,1.155438,0,1.329590,0.889894,0.928825,1.219812e+06,3.609838,1.568172,1.204130,1.778188,...,2.833694,6,2,99685.375000,495748.250000,128011.968750,98597.164062,0.995519,0.943437,0.001535
1,-0.880768,1,0.463217,0.414457,0.788038,5.954846e+05,0.414457,2.340907,0.619072,2.925315,...,2.171907,5,1,73758.687500,225793.921875,68017.406250,42586.082031,0.636021,1.003987,0.001450
2,0.212666,1,0.168891,1.067585,1.043223,7.060143e+05,3.105180,1.730127,0.658229,1.766358,...,2.189412,4,1,110116.250000,236623.578125,126511.656250,47005.578125,0.960368,0.941945,0.001524
3,0.332677,2,1.323761,0.642110,0.488249,8.401949e+05,2.741835,2.741835,1.360282,2.360640,...,2.389086,5,1,56536.421875,353416.281250,138765.578125,85941.804688,0.989835,0.946505,0.001931
4,0.219985,3,1.913489,1.176604,1.057895,1.365021e+06,1.656937,1.927142,1.771835,1.254011,...,0.472404,7,1,83682.382812,543441.937500,176302.593750,157836.390625,0.993249,0.965123,0.001253


 * X = signal + background data;
 * object y - hold the class labels.

The correspondence between `X` and `y` is given by the index

In [4]:
X = np.concatenate((hd_ttH,hd_ttW)) # training data                                                                                                                               
y = np.concatenate((np.ones(hd_ttH.shape[0]),np.zeros(hd_ttW.shape[0]))) # class lables                                                                                           


In [6]:
X[:1]

array([[ 1.15543771e+00,  0.00000000e+00,  1.32959020e+00,
         8.89893949e-01,  9.28824604e-01,  1.21981150e+06,
         3.60983849e+00,  1.56817186e+00,  1.20412970e+00,
         1.77818847e+00,  5.54543984e+04,  0.00000000e+00,
         0.00000000e+00,  4.65391352e-08,  1.04127896e+00,
         9.68156219e-01,  9.65672076e-01,  1.00000000e+00,
         3.42600000e+03,  2.01600000e+03,  3.90201367e+04,
         6.51848711e+04,  1.32959020e+00,  2.04843211e+00,
         3.45005039e+04,  1.74152464e-01, -1.64732492e+00,
        -2.00000000e+00,  1.08046875e+05,  4.01829844e+04,
         2.83369398e+00,  6.00000000e+00,  2.00000000e+00,
         9.96853750e+04,  4.95748250e+05,  1.28011969e+05,
         9.85971641e+04,  9.95519280e-01,  9.43436801e-01,
         1.53505707e-03]])

In [12]:
y[:1]

array([1.])

## scikit-learn

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

Split sample to training and test in fraction: 2/3 train, 1/3 test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

define BDT model

In [15]:
bdt = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=4, max_features='auto', min_samples_split=10, min_samples_leaf=10), n_estimators=100, learning_rate=1)

train model

In [16]:
bdt.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=100, random_state=None)

get scores from test sample      

In [17]:
y_predicted = bdt.predict(X_test)

classification performance      

In [18]:
print( classification_report(y_test, y_predicted, target_names=["signal", "background"]))
print( "Area under ROC curve: %.4f"%(roc_auc_score(y_test, bdt.decision_function(X_test))))

              precision    recall  f1-score   support

      signal       0.99      0.99      0.99     28672
  background       0.99      0.99      0.99     23025

   micro avg       0.99      0.99      0.99     51697
   macro avg       0.99      0.99      0.99     51697
weighted avg       0.99      0.99      0.99     51697

Area under ROC curve: 0.9987
